In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AADHI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = nltk.corpus.stopwords.words('english')

In [4]:
print(stop_words[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


I am watching a movie and I like it - like , watching movie 

## Stopwords Removal

In [5]:
def remove_stopwords(text, is_lower_case = False, stopwords = None):
    if not stopwords: 
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case: 
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else: 
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_tokens = ' '.join(filtered_tokens)
    return filtered_tokens

In [6]:
remove_stopwords('The quick brown fox is jumping over the lazy dog', is_lower_case = False)

'quick brown fox jumping lazy dog'

In [7]:
s = 'The quick brown fox is jumping over the lazy dog!'
tokens = nltk.word_tokenize(s)

In [8]:
tokens

['The',
 'quick',
 'brown',
 'fox',
 'is',
 'jumping',
 'over',
 'the',
 'lazy',
 'dog',
 '!']

In [9]:
nltk.pos_tag(tokens)

[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('is', 'VBZ'),
 ('jumping', 'VBG'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog', 'NN'),
 ('!', '.')]

In [10]:
import re
from bs4 import BeautifulSoup
import unicodedata
import contractions
import spacy
import nltk
from nltk.stem import PorterStemmer
import en_core_web_sm
ps =PorterStemmer()

nlp = en_core_web_sm.load()
ps = nltk.porter.PorterStemmer()


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def simple_stemming(text, stemmer=ps):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [12]:
import tqdm

def text_pre_processor(text, html_strip=True, accented_char_removal=True, contraction_expansion=True,
                       text_lower_case=True, text_stemming=False, text_lemmatization=True, 
                       special_char_removal=True, remove_digits=True, stopword_removal=True, 
                       stopword_list=None):
    
    # strip HTML
    if html_strip:
        text = strip_html_tags(text)
    
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    if accented_char_removal:
        text = remove_accented_chars(text)
    
    # expand contractions    
    if contraction_expansion:
        text = expand_contractions(text)
        
    
    # lemmatize text
    if text_lemmatization:
        text = spacy_lemmatize_text(text) 
        
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub(" \\1 ", text)
        text = remove_special_characters(text, remove_digits=remove_digits)  
        
    # stem text
    if text_stemming and not text_lemmatization:
        text = simple_stemming(text)
        
    # lowercase the text    
    if text_lower_case:
        text = text.lower()
        
        
    # remove stopwords
    if stopword_removal:
        text = remove_stopwords(text, is_lower_case=text_lower_case, 
                                stopwords=stopword_list)
    
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
    
    
    return text

def corpus_pre_processor(corpus):
    norm_corpus = []
    for doc in tqdm.tqdm(corpus):
        norm_corpus.append(text_pre_processor(doc))
    return norm_corpus

In [ ]:
!pip install emot --upgrade
!pip instlall emo_unicode

In [14]:
!pip install emoji

  Running setup.py bdist_wheel for emoji: started
  Running setup.py bdist_wheel for emoji: finished with status 'done'
  Stored in directory: C:\Users\AADHI\AppData\Local\pip\Cache\wheels\2a\a9\0a\4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [15]:
import emoji 
def convert_emojis(text):
    return emoji.demojize(text).replace(':'," ").replace(",", " ")
    

In [16]:
convert_emojis("game is on 🔥🔥")

'game is on  fire  fire '

In [11]:
!pip install emot --upgrade
!pip instlall emo_unicode

You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
ERROR: unknown command "instlall" - maybe you meant "install"

